# Demonstration Recording Script

This is a script for recording multiple demonstrations based on an environment.

`task_variant` can be set to `rR` (rotation reduced) or `rP` (rotation plus).

It has some special functions that allow re-orientation of the objects, this is done with the `update_object_orn` funciton, which gets and modifies the object pose.

In [6]:
import os
import time
import json
import shutil
import unittest
import subprocess
from pathlib import Path

from scipy.spatial.transform import Rotation as R

from gym_grasping.envs.robot_sim_env import RobotSimEnv
from flow_control.demo.record_sim import record_sim
from flow_control.servoing.runner import evaluate_control
from flow_control.servoing.module import ServoingModule

root_dir = "./tmp/flow_experiments3_pnp"
# root_dir = "/home/argusm/CLUSTER/flow_dataset"
renderer = "debug"
task="recombination"
object_selected = "trapeze" # trapeze
task_variant = "rP"  # rotation plus (+-pi)

def get_configurations(root_dir=root_dir, num_episodes=40, prefix=""):
    os.makedirs(root_dir, exist_ok=True)
    save_dir_template = os.path.join(root_dir, f"{prefix}_{task}_{object_selected}")
    for seed in range(num_episodes):
        seed = seed
        print(seed)
        save_dir = save_dir_template + f"_{task_variant}"+f"_seed{seed:03d}"
        yield object_selected, task_variant, seed, save_dir

# Record Episodes


In [ ]:
from math import pi

# def update_object_orn(env, object_selected, new_orn):
#     object_uid = env._task.object_uids[object_selected]
#     object_pos, object_orn = env.p.getBasePositionAndOrientation(object_uid)
#     print("pos & orn", object_pos, object_orn)
    
#     env.p.resetBasePositionAndOrientation(object_uid, object_pos, new_orn,
#                                           physicsClientId=env.cid)

#     object_pos, object_orn = env.p.getBasePositionAndOrientation(object_uid)
#     print("pos & orn2", object_pos, object_orn)

demo_cfgs = get_configurations(prefix="demo")
for object_selected, task_variant, seed, save_dir in demo_cfgs:
    param_info = {"object_selected": object_selected}
    env = RobotSimEnv(task='shape_sorting', renderer=renderer, act_type='continuous',
                      initial_pose='close', max_steps=200, control='absolute-full',
                      img_size=(256, 256),
                      param_randomize=("geom",),
                      param_info=param_info,
                      task_info=dict(object_rot_range={"rP":pi/2.,"rR":pi/6.}[task_variant]),
                      seed=seed)
    
    if task_variant == "rP":
        assert env.params.variables[f"{object_selected}_pose"]["d"][3] == pi/2.
    elif task_variant == "rR":
        assert env.params.variables[f"{object_selected}_pose"]["d"][3] == pi/6.
    #update_object_orn(env, object_selected, orn)
        
    if os.path.isdir(save_dir):
        # lsof file if there are NSF issues.
        shutil.rmtree(save_dir)
    record_sim(env, save_dir)
    
    del env
    time.sleep(.5)
    print(save_dir)

In [ ]:
from tqdm import tqdm
# Convert notebook to script
convert_cmd = "jupyter nbconvert --to script ../demo_segment/Segment_Color.ipynb"
convert_cmd = convert_cmd.split()
subprocess.run(convert_cmd, check=True)

for _, _, seed, save_dir in tqdm(get_configurations(prefix="demo")):
    segment_cmd = f"python ../demo_segment/Segment_Color.py {save_dir}"
    subprocess.run(segment_cmd.split(), check=False)

# Cleanup, don't leave file lying around because e.g. github PEP check
os.remove("../demo_segment/Segment_Color.py")

## Show Recorded Demos. 

In [ ]:
from flow_control.demo.playback_env_servo import PlaybackEnvServo

demo_cfgs = get_configurations(prefix="demo")
recordings = []
for _, _, demo_seed, demo_dir in demo_cfgs:
    recordings.append(demo_dir)
    
print("Number of recordings:", len(recordings))
print(recordings[0])
print(recordings[-1])
# Load the demonstration episodes
playbacks = [PlaybackEnvServo(rec) for rec in recordings[:]]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Layout

# Plot the demonstrations
%matplotlib notebook
fig, ax = plt.subplots(1,figsize=(8, 6))
fig.suptitle("Demonstration Frames")
ax.set_axis_off()
image_h = ax.imshow(playbacks[0].cam.get_image()[0])

def update(demo_index, frame_index):
    image = playbacks[demo_index][frame_index].cam.get_image()[0]
    image_h.set_data(image)
    fig.canvas.draw_idle()
    print("wp_name:", playbacks[demo_index][frame_index].get_info()["wp_name"])
    print(playbacks[demo_index][frame_index].get_info()["move_anchor"])
    
    fg_mask = playbacks[demo_index].get_fg_mask()
    if fg_mask is not None:
        print("percent fg @ 0:", np.mean(fg_mask)*100)
    
slider_w = widgets.IntSlider(min=0, max=len(playbacks)-1, step=1, value=0,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0, max=200-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_index=slider_w, frame_index=slider_i)

In [ ]:
def filter_demo(pb):
    return pb[-1].data['rew'] > 0 and np.mean(pb.get_fg_mask()) > 0.005

demo_good = [filter_demo(pb) for pb in playbacks]
good_demonstrations = np.where(demo_good)[0]
print(good_demonstrations)
good_demonstrations = [int(x) for x in good_demonstrations]

#print(np.array(demo_good).astype(int))

## Segment Demonstration into Parts

This is done via the waypoint names, which have the following format `<block-name>_<detail-descr>`.
We group `block-name` together and apply block_name_map to group blocks.

In [ ]:
from itertools import tee
block_name_map, parts_name = {'start': 'locate'}, "manual3"
#block_name_map, parts_name = {'start': 'locate', 'grasp': 'locate'}, "manual2"

def get_block_name(wp_name, block_map):
    block_name = wp_name.split("_")[0]
    if block_name in block_map:
        return block_map[block_name]
    return block_name

def pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

demo_parts = {}
for demo_index in good_demonstrations:
    wp_names = []
    for frame_index in range(len(playbacks[demo_index])-1):
        wp_name = playbacks[demo_index][frame_index].get_info()["wp_name"]
        wp_names.append(wp_name)
        
    block_names = np.array([get_block_name(wp, block_name_map) for wp in wp_names])
    transitions = np.where(block_names[1:] != block_names[:-1])[0]
    print(transitions)
    for i in range(len(transitions)+1):
        if i == 0:
            pass
    trn_name = [*block_names[transitions],  block_names[transitions[-1]+1]]
    trn_idx = pairwise([0,*transitions, len(playbacks[demo_index])-1])
    
    ep_demo_parts = []
    for name, trn in zip(trn_name, trn_idx):
        ep_demo_parts.append(dict(name=name, start=int(trn[0]), end=int(trn[1])))
    demo_parts[demo_index] = ep_demo_parts
    print(ep_demo_parts)

print(f"Demo Parts (demo={demo_index}):")
for e in ep_demo_parts:
    print(f"{e['name']}:\t{e['start']:02d}->{e['end']:02d}")

demo_parts_fn = os.path.join(root_dir, f"demo_parts_{parts_name}.json")
with open(demo_parts_fn, "w") as f_obj:
    json.dump(demo_parts, f_obj)
    
print("saved demo parts to :", demo_parts_fn)

In [ ]:
# Plot the end of segments
%matplotlib notebook
max_ep_parts = max([len(ep_parts) for ep_parts in demo_parts.values()])

fig, (ax, ax2) = plt.subplots(1,2,figsize=(8, 6))
fig.suptitle("Show Demonstration Parts")
ax.set_axis_off()
ax2.set_axis_off()
image_h = ax.imshow(playbacks[0].cam.get_image()[0])
image_h2 = ax2.imshow(playbacks[0].cam.get_image()[0])

max_frames = max([len(b) for b in demo_parts.values()])
def update(demo_ep, part_idx):
    demo_index = list(demo_parts.keys())[demo_ep]
    demo_name = demo_parts[demo_index][part_idx]["name"]
    start_index = demo_parts[demo_index][part_idx]["start"]
    end_index = demo_parts[demo_index][part_idx]["end"]
    ax.set_title(f"{demo_name}: start @ {demo_index}, {start_index}")
    ax2.set_title(f"{demo_name}: end @ {demo_index}, {end_index}")
    
    image = playbacks[demo_index][start_index].cam.get_image()[0]
    image_h.set_data(image)
    image2 = playbacks[demo_index][end_index].cam.get_image()[0]
    image_h2.set_data(image2)
    fig.canvas.draw_idle()

    print("wp_name:", playbacks[demo_index][start_index].get_info()["wp_name"])
    
slider_w = widgets.IntSlider(min=0, max=len(demo_parts)-1, step=1, value=0,
                             layout=Layout(width='70%'))

slider_i = widgets.IntSlider(min=0, max=max_ep_parts-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_ep=slider_w, part_idx=slider_i)
plt.tight_layout()
plt.show()

### Check if the block size is still randomized

In [7]:
import os
import time
import json
import shutil
import unittest
import subprocess
from pathlib import Path

from scipy.spatial.transform import Rotation as R
from math import pi
# from gym_grasping.envs.robot_sim_env import RobotSimEnv
# from flow_control.demo.demo_episode_recorder import record_sim
# from flow_control.runner import evaluate_control
# from flow_control.servoing.module import ServoingModule
# from math import pi
# import getpass

# experiment = "multi_task_1_vs_n"
# goal = "multi_task_goal_1_vs_n"

# def get_data_dir():
#     username = getpass.getuser()
#     if username == "argusm":
#         return "/tmp/flow_experiments3"
#     elif username == "nayakab":
#         return "./tmp"

# data_dir = get_data_dir()

# root_dir = os.path.join(data_dir, experiment)
# goal_dir = os.path.join(data_dir, goal)

root_dir = './tmp_flowexperiments3_pnp_test'

renderer = "debug"
object_selected = "trapeze"
task_variant = "rP"  # rotation plus (+-pi)

def get_configurations(root_dir=root_dir, task="shape_sorting", num_episodes=20, start_seed=0, prefix="", object_selected='trapeze'):
    os.makedirs(root_dir, exist_ok=True)
    save_dir_template = os.path.join(root_dir, f"{prefix}_{task}_{object_selected}")
    for seed in range(start_seed, start_seed + num_episodes):
        save_dir = save_dir_template + f"_{task_variant}"+f"_seed{seed:03d}"
        yield object_selected, task_variant, seed, save_dir


In [8]:
cfg = { 0: {'prefix': 'demo', 'task': 'pick_n_place', 'num_episodes': 20, 'start_seed': 40, 'object_selected': object_selected, 'root_dir': root_dir}}

for key, value in cfg.items():
    demo_cfg = get_configurations(value['root_dir'], value['task'], 
                                  value['num_episodes'], value['start_seed'],
                                  prefix=value['prefix'],
                                  object_selected=value['object_selected'])
    for object_selected, task_variant, seed, save_dir in demo_cfg:
        param_info = {"object_selected": value['object_selected'], "task_selected": value['task']}
        env = RobotSimEnv(task='recombination', renderer=renderer, act_type='continuous',
                          initial_pose='close', max_steps=200, control='absolute-full',
                          img_size=(256, 256),
                          param_randomize=("geom",),
                          param_info=param_info,
                          task_info=dict(object_rot_range={"rP":pi/2.,"rR":pi/6.}[task_variant]),
                          seed=seed)

        if task_variant == "rP":
            assert env.params.variables[f"{object_selected}_pose"]["d"][3] == pi/2.
        elif task_variant == "rR":
            assert env.params.variables[f"{object_selected}_pose"]["d"][3] == pi/6.
        #update_object_orn(env, object_selected, orn)

        if os.path.isdir(save_dir):
            # lsof file if there are NSF issues.
            shutil.rmtree(save_dir)
        record_sim(env, save_dir)

        del env
        time.sleep(.5)
        print(save_dir)


INFO - 2023-05-16 13:24:46,734 - robot_sim_env - Calibration Time:2020-10-28 17:26:16.330854
INFO - 2023-05-16 13:24:46,845 - env_param_sampler - param table_size changed
INFO - 2023-05-16 13:24:46,846 - env_param_sampler - param table_pos changed
INFO - 2023-05-16 13:24:46,846 - env_param_sampler - param table_orn changed
INFO - 2023-05-16 13:24:46,846 - env_param_sampler - param object_size changed
INFO - 2023-05-16 13:24:46,847 - env_param_sampler - param object_to_tcp changed
INFO - 2023-05-16 13:24:46,847 - env_param_sampler - param block_red changed
INFO - 2023-05-16 13:24:46,847 - env_param_sampler - param block_blue changed
/home/argusm/miniconda3/envs/ur_env/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
INFO - 2023-05-16 13:24:46,849 - env_param_sampler - param max_steps: 50 -> 200
INFO - 2023-05-16 13:24:48,345 - recombination_task 

Recording ended with reward:  1.0


INFO - 2023-05-16 13:24:49,748 - simple_recorder - saved recording to: ./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed040 w/ length 37
INFO - 2023-05-16 13:24:51,049 - robot_sim_env - Calibration Time:2020-10-28 17:26:16.330854
INFO - 2023-05-16 13:24:51,180 - env_param_sampler - param table_size changed
INFO - 2023-05-16 13:24:51,180 - env_param_sampler - param table_pos changed
INFO - 2023-05-16 13:24:51,181 - env_param_sampler - param table_orn changed
INFO - 2023-05-16 13:24:51,181 - env_param_sampler - param object_size changed
INFO - 2023-05-16 13:24:51,181 - env_param_sampler - param object_to_tcp changed
INFO - 2023-05-16 13:24:51,182 - env_param_sampler - param block_red changed
INFO - 2023-05-16 13:24:51,182 - env_param_sampler - param block_blue changed
INFO - 2023-05-16 13:24:51,183 - env_param_sampler - param max_steps: 50 -> 200


./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed040


INFO - 2023-05-16 13:24:52,665 - recombination_task - Completed wp: locate_high
INFO - 2023-05-16 13:24:52,765 - recombination_task - Completed wp: locate_mid
INFO - 2023-05-16 13:24:52,965 - recombination_task - Completed wp: grasp_locate
INFO - 2023-05-16 13:24:53,199 - recombination_task - Completed wp: grasp
INFO - 2023-05-16 13:24:53,366 - recombination_task - Completed wp: insert_height
INFO - 2023-05-16 13:24:53,583 - recombination_task - Completed wp: insert_over


...

Recording ended with reward:  1.0


INFO - 2023-05-16 13:26:07,847 - robot_sim_env - Calibration Time:2020-10-28 17:26:16.330854
INFO - 2023-05-16 13:26:07,970 - env_param_sampler - param table_size changed
INFO - 2023-05-16 13:26:07,970 - env_param_sampler - param table_pos changed
INFO - 2023-05-16 13:26:07,971 - env_param_sampler - param table_orn changed
INFO - 2023-05-16 13:26:07,971 - env_param_sampler - param object_size changed
INFO - 2023-05-16 13:26:07,971 - env_param_sampler - param object_to_tcp changed
INFO - 2023-05-16 13:26:07,972 - env_param_sampler - param block_red changed
INFO - 2023-05-16 13:26:07,972 - env_param_sampler - param block_blue changed
INFO - 2023-05-16 13:26:07,973 - env_param_sampler - param max_steps: 50 -> 200


./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed058


INFO - 2023-05-16 13:26:09,462 - recombination_task - Completed wp: locate_high
INFO - 2023-05-16 13:26:09,529 - recombination_task - Completed wp: locate_mid
INFO - 2023-05-16 13:26:09,662 - recombination_task - Completed wp: grasp_locate
INFO - 2023-05-16 13:26:09,896 - recombination_task - Completed wp: grasp
INFO - 2023-05-16 13:26:10,046 - recombination_task - Completed wp: insert_height
INFO - 2023-05-16 13:26:10,279 - recombination_task - Completed wp: insert_over
INFO - 2023-05-16 13:26:10,713 - simple_recorder - saved recording to: ./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed059 w/ length 34


Recording ended with reward:  1.0
./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed059


In [9]:
from tqdm import tqdm
# Convert notebook to script
convert_cmd = "jupyter nbconvert --to script ../demo_segment/Segment_Color.ipynb"
convert_cmd = convert_cmd.split()
subprocess.run(convert_cmd, check=True)

for key, value in cfg.items():
    demo_cfg = get_configurations(value['root_dir'], value['task'], 
                                  value['num_episodes'], value['start_seed'],
                                  prefix=value['prefix'],
                                  object_selected=value['object_selected'])
    for object_selected, task_variant, seed, save_dir in demo_cfg:
        param_info = {"object_selected": value['object_selected'], "task_selected": value['task']}
        segment_cmd = f"python ../demo_segment/Segment_Color.py {save_dir}"
        
        subprocess.run(segment_cmd.split(), check=False)
        

# Cleanup, don't leave file lying around because e.g. github PEP check
os.remove("../demo_segment/Segment_Color.py")

[NbConvertApp] Converting notebook ../demo_segment/Segment_Color.ipynb to script
[NbConvertApp] Writing 27242 bytes to ../demo_segment/Segment_Color.py
INFO - 2023-05-16 13:26:19,390 - Segment_Color - Trajectory segmentation method: waypoint-names


Usage: Segment_Color.py <episode_dir>
loaded segmentation masks 37/37
loaded move_anchors 37/37
loaded waypoint names 37/37
loaded.
Skipping keyframe @ 19: gripper closing grasp_close
Removing keyframe @ 12: too close to 13

fr.#  name          servo   trj-act    grip_dist
--------------------------------------------------
0     start         servo   abs        3         
4     locate_high           rel        2         
7     locate_mid    servo   rel        1         
13    gripper_close servo   rel        0         
24    insert_height servo   rel->rel   2         
29    insert_over   servo   abs        1         
33    gripper_open  servo   rel        0         
36    demo_end              rel->rel   10        


Saved to ./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed040/servo_keep.json
Skipping color segmentation, config file not found:
./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed040/segment_conf.json
Saved to ./tmp_flowexperiments3_pnp_test

INFO - 2023-05-16 13:26:21,653 - Segment_Color - Trajectory segmentation method: waypoint-names


Usage: Segment_Color.py <episode_dir>
loaded segmentation masks 39/39
loaded move_anchors 39/39
loaded waypoint names 39/39
loaded.
Skipping keyframe @ 21: gripper closing grasp_close
Removing keyframe @ 14: too close to 15

fr.#  name          servo   trj-act    grip_dist
--------------------------------------------------
0     start         servo   abs        3         
5     locate_high           rel        2         
8     locate_mid    servo   rel        1         
15    gripper_close servo   rel        0         
26    insert_height servo   rel->rel   2         
31    insert_over   servo   abs        1         
35    gripper_open  servo   rel        0         
38    demo_end              rel->rel   10        


Saved to ./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed041/servo_keep.json
Skipping color segmentation, config file not found:
./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed041/segment_conf.json
Saved to ./tmp_flowexperiments3_pnp_test

INFO - 2023-05-16 13:26:23,897 - Segment_Color - Trajectory segmentation method: waypoint-names


...

Usage: Segment_Color.py <episode_dir>
loaded segmentation masks 35/35
loaded move_anchors 35/35
loaded waypoint names 35/35
loaded.
Skipping keyframe @ 19: gripper closing grasp_close
Removing keyframe @ 12: too close to 13

fr.#  name          servo   trj-act    grip_dist
--------------------------------------------------
0     start         servo   abs        3         
5     locate_high           rel        2         
8     locate_mid    servo   rel        1         
13    gripper_close servo   rel        0         
23    insert_height servo   rel->rel   2         
27    insert_over   servo   abs        1         
31    gripper_open  servo   rel        0         
34    demo_end              rel->rel   10        


Saved to ./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed057/servo_keep.json
Skipping color segmentation, config file not found:
./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed057/segment_conf.json
Saved to ./tmp_flowexperiments3_pnp_test

INFO - 2023-05-16 13:26:59,595 - Segment_Color - Trajectory segmentation method: waypoint-names


Usage: Segment_Color.py <episode_dir>
loaded segmentation masks 36/36
loaded move_anchors 36/36
loaded waypoint names 36/36
loaded.
Skipping keyframe @ 20: gripper closing grasp_close
Removing keyframe @ 13: too close to 14

fr.#  name          servo   trj-act    grip_dist
--------------------------------------------------
0     start         servo   abs        3         
5     locate_high           rel        2         
8     locate_mid    servo   rel        1         
14    gripper_close servo   rel        0         
25    insert_height servo   rel->rel   2         
30    insert_over   servo   abs        1         
34    gripper_open  servo   rel        0         
35    demo_end              rel->rel   10        


Saved to ./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed058/servo_keep.json
Skipping color segmentation, config file not found:
./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed058/segment_conf.json
Saved to ./tmp_flowexperiments3_pnp_test

INFO - 2023-05-16 13:27:01,823 - Segment_Color - Trajectory segmentation method: waypoint-names


Usage: Segment_Color.py <episode_dir>
loaded segmentation masks 34/34
loaded move_anchors 34/34
loaded waypoint names 34/34
loaded.
Auto-detected: non-continous trajectory
Skipping keyframe @ 17: gripper closing grasp_close
Removing keyframe @ 10: too close to 11

fr.#  name          servo   trj-act    grip_dist
--------------------------------------------------
0     start         servo   abs        3         
4     locate_high           rel        2         
6     locate_mid    servo   rel        1         
11    gripper_close servo   rel        0         
21    insert_height servo   rel->rel   2         
26    insert_over   servo   abs        1         
30    gripper_open  servo   rel        0         
33    demo_end              rel->rel   10        


Saved to ./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed059/servo_keep.json
Skipping color segmentation, config file not found:
./tmp_flowexperiments3_pnp_test/demo_pick_n_place_trapeze_rP_seed059/segment_conf.json
